# Classification

### Intro

I have designed three classifiers: a naive Bayes classifier, a perceptron classifier and a large-margin (MIRA) classifier. The classifiers are tested on two image data sets: a set of scanned handwritten digit images and a set of face images in which edges have already been detected. Even with simple features, the classifiers are able to do quite well on these tasks when given enough training data.

Face detection is the task of localizing faces within video or still images. The faces can be at any location and vary in size. There are many applications for face detection, including human computer interaction and surveillance. 
Shown here is a simplified face detection task in which my system is presented with an image that has been pre-processed by an edge detection algorithm. My system is then tasked to determine whether the edge image is a face or not.

### Let's get started

Let's try our classification pipeline by running dataClassifier.py from the command line. This will classify the digit data using the default classifier (mostFrequent) which blindly classifies every example with the most frequent label.

In [4]:
!python2 dataClassifier.py -h

Usage: 
  USAGE:      python dataClassifier.py <options>
  EXAMPLES:   (1) python dataClassifier.py
                  - trains the default mostFrequent classifier on the digit dataset
                  using the default 100 training examples and
                  then test the classifier on test data
              (2) python dataClassifier.py -c naiveBayes -d digits -t 1000 -f -o -1 3 -2 6 -k 2.5
                  - would run the naive Bayes classifier on 1000 training examples
                  using the enhancedFeatureExtractorDigits function to get the features
                  on the faces dataset, would use the smoothing parameter equals to 2.5, would
                  test the classifier on the test data and performs an odd ratio analysis
                  with label1=3 vs. label2=6
                 

Options:
  -h, --help            show this help message and exit
  -c CLASSIFIER, --classifier=CLASSIFIER
                        The type of classifier [Default: mostFrequent]
  -

In [2]:
!python2 dataClassifier.py  

Doing classification
--------------------
data:		digits
classifier:		mostFrequent
using enhanced features?:	False
training set size:	100
Extracting features...
Training...
Validating...
14 correct out of 100 (14.0%).
Testing...
14 correct out of 100 (14.0%).
Predicted 1; truth is 9
Image: 
                            
                            
                            
                            
                            
                            
                            
             ++###+         
             ######+        
            +######+        
            ##+++##+        
           +#+  +##+        
           +##++###+        
           +#######+        
           +#######+        
            +##+###         
              ++##+         
              +##+          
              ###+          
            +###+           
            +##+            
           +##+             
          +##+              
         +##+               
         ##+ 

This classifier comes with some pre defined simple features. However, we are going to see how to design some better features.

Currently, the simple feature set includes one feature for each pixel location, which can take values 0 or 1 (off or on). The features are encoded as a Counter where keys are feature locations (represented as (column,row)) and values are 0 or 1. The face recognition data set has value 1 only for those pixels identified by a Canny edge detector.

In [18]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

## Naive Bayes

Let's start by implementing a naive Bayes classifier.

### Theory 

A naive Bayes classifier models a joint distribution over a label $Y$ and a set of observed random variables, or features,  $\{F_1, F_2, \ldots F_n\}$, using the assumption that the full joint distribution can be factored as follows (features are conditionally independent given the label): 

$$P\left(F_{1}, \ldots, F_{n}, Y\right)=P(Y) \prod_{i} P\left(F_{i} | Y\right)$$

To classify a datum, we can find the most probable label given the feature values for each pixel, using Bayes theorem:

$$\begin{aligned} P\left(y | f_{1}, \ldots, f_{m}\right) &=\frac{P\left(f_{1}, \ldots, f_{m} | y\right) P(y)}{P\left(f_{1}, \ldots, f_{m}\right)} \\ &=\frac{P(y) \prod_{i=1}^{m} P\left(f_{i} | y\right)}{P\left(f_{1}, \ldots, f_{m}\right)} \\ \arg \max _{y} P\left(y | f_{1}, \ldots, f_{m}\right) &=\arg \max _{y} \frac{P(y) \prod_{i=1}^{m} P\left(f_{i} | y\right)}{P\left(f_{1}, \ldots, f_{m}\right)} \\ &=\arg \max _{y} P(y) \prod_{i=1}^{m} P\left(f_{i} | y\right) \end{aligned}$$

Because multiplying many probabilities together often results in underflow, we will instead compute log probabilities which have the same argmax:

$$\begin{aligned} \arg \max _{y} \log P\left(y | f_{1}, \ldots, f_{m}\right) &=\arg \max _{y} \log P\left(y, f_{1}, \ldots, f_{m}\right) \\ &=\arg \max _{y}\left\{\log P(y)+\sum_{i=1}^{m} \log P\left(f_{i} | y\right)\right\} \end{aligned}$$


### Parameter Estimation

Our naive Bayes model has several parameters to estimate. One parameter is the prior distribution over labels (digits, or face/not-face), $P(Y)$.
We can estimate $P(Y)$ directly from the training data: 
$$\hat{P}(y)=\frac{c(y)}{n}$$

where $c(y)$ is the number of training instances with label y and n is the total number of training instances.
The other parameters to estimate are the conditional probabilities of our features given each label y: $P(F_i \vert Y = y)$. We do this for each possible feature value $\left(f_{i} \in\{0,1\}\right)$.

$$\hat{P}\left(F_{i}=f_{i} | Y=y\right)=\frac{c\left(f_{i}, y\right)}{\sum_{f_{i}^{\prime} \in\{0,1\}} c\left(f_{i}^{\prime}, y\right)}$$

where $c(f_i,y)$ is the number of times pixel $F_i$ took value $f_i$ in the training examples of label y.

### Smoothing

The current parameter estimates are unsmoothed, that is, you are using the empirical estimates for the parameters $P(f_i\vert y)$. These estimates are rarely adequate in real systems. Minimally, we need to make sure that no parameter ever receives an estimate of zero, but good smoothing can boost accuracy quite a bit by reducing overfitting.
In this project, we use Laplace smoothing, which adds k counts to every possible observation value:

$$P\left(F_{i}=f_{i} | Y=y\right)=\frac{c\left(f_{i}, y\right)+k}{\sum_{i_{i}^{\prime} \in\{0,1\}}\left(c\left(f_{i}^{\prime}, y\right)+k\right)}$$

If k=0, the probabilities are unsmoothed. As k grows larger, the probabilities are smoothed more and more. You can use your validation set to determine a good value for k. Note: don't smooth P(Y).

### Implementation 

We'll implement a trainAndTune function that trains the classifier by collecting counts over the training data, and
stores the Laplace smoothed estimates so that they can be used to classify. 

Next, we'll implement a calculateLogJointProbabilities in our naiveBayes.py file. In our trainAndTune function, we'll estimate conditional probabilities from the training data for each possible value of k given in the list kgrid. Then we'll evaluate accuracy on the held-out validation set for each k and choose the value with the highest validation accuracy. In case of ties, prefer the lowest value of k.

We can now test our classifier with the following command:

Implementation Note: You'll find it easiest to hard-code the binary feature assumption. If you do, make sure you don't include any non-binary features. Or, you can write your code more generally, to handle arbitrary feature values, though this will probably involve a preliminary pass through the training set to find all possible feature values (and you'll need an "unknown" option in case you encounter a value in the test data you never saw during training).